## Bank Statement Uploader Notebook

This notebook allows you to upload a bank statement (in `.csv` or `.xlsx` format) and saves the transactions into a local SQLite database (`transactions.db`).

**Instructions:**
1. Run the cells in order.
2. When you run the final cell, an "Upload" button will appear.
3. Click the button, select your bank statement file, and the transactions will be processed and saved automatically.

In [7]:
# Step 1: Import necessary libraries
import pandas as pd
import sqlite3
import ipywidgets as widgets
from IPython.display import display
import io

### Step 2: Setup Database Connection
This cell connects to the `transactions.db` file and creates the `transactions` table if it doesn't already exist.

In [8]:
db_name = 'transactions.db'
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

cursor.execute("""
                CREATE TABLE IF NOT EXISTS transactions (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    transaction_date TEXT not null,
                    description TEXT,
                    debit_amount REAL,
                    credit_amount REAL,
                    balance REAL
                )
            """)
conn.commit()
conn.close()

print(f"Database '{db_name}' is ready.")

Database 'transactions.db' is ready.


### Step 3: Define Processing Functions
These are the core functions that parse the bank statement and save the data to the database.

In [9]:
def parse_statement(df):
    """
    Detects the bank format and maps its columns to the standard database schema.
    """
    df.columns = df.columns.str.strip()

    bank_a_mapping = {
        'Date': 'transaction_date',
        'Narration': 'description',
        'Debit': 'debit_amount',
        'Credit': 'credit_amount',
        'Closing Balance': 'balance'
    }

    bank_b_mapping = {
        'Transaction Date': 'transaction_date',
        'Transaction Remarks': 'description',
        'Withdrawal Amount (INR)': 'debit_amount',
        'Deposit Amount (INR)': 'credit_amount',
        'Balance (INR)': 'balance'
    }

    if all(col in df.columns for col in bank_a_mapping.keys()):
        df_mapped = df.rename(columns=bank_a_mapping)
        target_cols = list(bank_a_mapping.values())
    elif all(col in df.columns for col in bank_b_mapping.keys()):
        df_mapped = df.rename(columns=bank_b_mapping)
        target_cols = list(bank_b_mapping.values())
    else:
        return None

    df_final = df_mapped[target_cols].copy()
    df_final['transaction_date'] = pd.to_datetime(df_final['transaction_date'], dayfirst=True, errors='coerce').dt.strftime('%Y-%m-%d')
    for col in ['debit_amount', 'credit_amount', 'balance']:
        df_final[col] = pd.to_numeric(df_final[col], errors='coerce')

    df_final['debit_amount'].fillna(0, inplace=True)
    df_final['credit_amount'].fillna(0, inplace=True)
    df_final['description'].fillna('', inplace=True)
    df_final.dropna(subset=['transaction_date', 'balance'], inplace=True)

    return list(df_final.itertuples(index=False, name=None))

def save_to_db(transactions):
    """
    Saves a list of transaction records to the SQLite database, skipping duplicates.
    A transaction is considered a duplicate if another entry has the same date, description, debit, and credit amount.
    Returns a tuple of (new_records_count, duplicate_records_count).
    """
    new_records_count = 0
    duplicate_records_count = 0
    records_to_insert = []

    try:
        conn = sqlite3.connect(db_name)
        cursor = conn.cursor()

        check_query = '''
            SELECT 1 FROM transactions
            WHERE transaction_date = ? AND description = ? AND debit_amount = ? AND credit_amount = ?
            LIMIT 1
        '''

        for t in transactions:
            # Check for duplicates using the first 4 values of the tuple
            cursor.execute(check_query, (t[0], t[1], t[2], t[3]))
            if cursor.fetchone():
                duplicate_records_count += 1
            else:
                records_to_insert.append(t)

        if records_to_insert:
            insert_query = '''
                INSERT INTO transactions (transaction_date, description, debit_amount, credit_amount, balance)
                VALUES (?, ?, ?, ?, ?)
            '''
            cursor.executemany(insert_query, records_to_insert)
            conn.commit()
            new_records_count = len(records_to_insert)

        conn.close()
        return new_records_count, duplicate_records_count
    except sqlite3.Error as e:
        # In case of a database error, we print it to the log.
        print(f"❌ Database Error: {e}")
        return 0, 0

### Step 4: Create the File Uploader Widget
This is the interactive part. An upload button will be created. When you upload a file, the `on_upload_change` function will trigger, process the file, and save the data.

In [10]:
# Create a widget to upload a file
uploader = widgets.FileUpload(
    accept='.csv,.xlsx',
    multiple=False,  # Set to True to allow multiple files
    description='Upload Statement'
)

# Create an output widget to display messages
output_log = widgets.Output()

def on_upload_change(change):
    """This function runs automatically when a file is uploaded."""
    # Direct all print statements within this function to the output_log widget
    with output_log:
        output_log.clear_output(wait=True)
        
        # Check if the file upload is cleared, if so, do nothing
        if not change['new']:
            return

        uploaded_file = change['new'][0]
        file_name = uploaded_file['name']
        content = uploaded_file['content']
        
        print(f"Processing '{file_name}'...")
        try:
            # Read the file content into a pandas DataFrame
            if file_name.endswith('.csv'):
                df = pd.read_csv(io.BytesIO(content))
            else:
                df = pd.read_excel(io.BytesIO(content))

            # Process the data
            transactions = parse_statement(df)
            if transactions:
                new_count, duplicate_count = save_to_db(transactions)
                print(f"✅ Processing complete for '{file_name}'.")
                print(f"   - Imported {new_count} new transactions.")
                print(f"   - Skipped {duplicate_count} duplicate records.")
            else:
                print(f"⚠️ Warning: Could not recognize the format of '{file_name}'.")
        
        except Exception as e:
            print(f"❌ Error processing '{file_name}': {e}")
            
    # Reset the uploader outside the 'with' block to ensure it always runs
    # This allows you to upload the same file again for testing if needed
    uploader.value = ()
    uploader._counter = 0


# Link the function to the uploader's value property
uploader.observe(on_upload_change, names='value')

# Display the uploader and the output log for messages
display(uploader, output_log)

FileUpload(value=(), accept='.csv,.xlsx', description='Upload Statement')

Output()

### Step 5: Database Cleanup Utility
If you uploaded files before the duplicate check was added, your database might have duplicate entries. Run the cell below to clean them up.

This script keeps the *first* instance of each unique transaction and deletes all subsequent duplicates. 

In [11]:
def clean_duplicates():
    try:
        conn = sqlite3.connect(db_name)
        cursor = conn.cursor()

        # First, count the total rows before cleaning
        cursor.execute('SELECT COUNT(*) FROM transactions')
        initial_count = cursor.fetchone()[0]

        # This query finds the first-occuring `id` for each unique combination of transaction fields
        # and deletes any rows whose `id` is NOT in that list of first-occurences.
        delete_query = '''
            DELETE FROM transactions
            WHERE id NOT IN (
                SELECT MIN(id)
                FROM transactions
                GROUP BY transaction_date, description, debit_amount, credit_amount
            )
        '''
        
        cursor.execute(delete_query)
        conn.commit()

        # Count the rows after cleaning to see how many were removed
        cursor.execute('SELECT COUNT(*) FROM transactions')
        final_count = cursor.fetchone()[0]
        
        conn.close()

        removed_count = initial_count - final_count
        if removed_count > 0:
            print(f"✅ Cleanup complete. Removed {removed_count} duplicate transactions.")
        else:
            print("✅ Database is already clean. No duplicates found.")

    except sqlite3.Error as e:
        print(f"❌ An error occurred during cleanup: {e}")

# Run the cleanup function
clean_duplicates()

✅ Database is already clean. No duplicates found.
